# 安裝套件

In [ ]:
# 套件安裝指令
!pip install --extra-index-url https://download.pytorch.org/whl/cu125 torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 accelerate==1.8.1 transformers datasets evaluate scikit-learn google.colab pandas

# 掛載 Google Drive

In [ ]:
# 會需要使用者同意授權/存取 Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# 切換目錄 (Colab 預設目錄為 /content，使用 %cd 切換目錄)
%cd /content/drive/MyDrive/

# 觀看系統設定

In [ ]:
!lsb_release -a

In [ ]:
!nvidia-smi

In [ ]:
!nvcc -V

In [ ]:
!python -V

# 微調模型

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0))

# 匯入套件

In [ ]:
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)

import random
import json
import pandas as pd
from sklearn.metrics import f1_score

# 基本參數設定

In [ ]:
model_name = 'google-bert/bert-base-chinese' # 預訓練模型名稱
max_seq_length = 512 # 可訓練的序列最大長度
num_labels = 8 # 多元分類 (8 種情緒)
output_dir = './output' # 輸出模型資料夾

# 載入 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 讀取 HuggingFace 資料集

In [ ]:
# 設定隨機種子，確保實驗可重現
seed = 42

# 從 HuggingFace 讀取資料集
df = pd.read_parquet("hf://datasets/Johnson8187/Chinese_Multi-Emotion_Dialogue_Dataset/data/train-00000-of-00001.parquet")

# 查看資料集基本資訊
print(f"資料集大小: {len(df)}")
print(f"欄位: {df.columns.tolist()}")
print(f"情緒類別: {df['emotion'].unique().tolist()}")
print(f"各類別數量:\n{df['emotion'].value_counts()}")

# 建立 label 對應表 (文字 -> 整數)
label_list = sorted(df['emotion'].unique().tolist())
label2id = {label: idx for idx, label in enumerate(label_list)}
id2label = {idx: label for label, idx in label2id.items()}

print(f"\nLabel 對應表:")
for label, idx in label2id.items():
    print(f"  {label} -> {idx}")

# 將文字 label 轉換為整數
df['labels'] = df['emotion'].map(label2id)

# 洗牌
df = df.sample(frac=1, random_state=seed).reset_index(drop=True)

sentences = df['text'].tolist()
labels = df['labels'].tolist()

# 轉換成 huggingface trainer 可以使用的 datasets

In [ ]:
# 建立 Dataset
dataset = Dataset.from_dict({
    'sentences': sentences,
    'labels': labels
})

# 回傳切分資料 (訓練 和 驗證)
dataset = dataset.train_test_split(test_size=0.2)
'''
print(dataset) 的內容如下:

DatasetDict({
    train: Dataset({
        features: ['sentences', 'labels'],
        num_rows: 6212
    })
    test: Dataset({
        features: ['sentences', 'labels'],
        num_rows: 1554
    })
})
'''

# 預處理資料
def preprocess_data(dataset):
    # 將句子轉換為 token (tokenization)
    return tokenizer(
        dataset['sentences'], 
        truncation=True, 
        padding=True, 
        return_tensors='pt', 
        max_length=max_seq_length
    )

# 轉換資料
train_data = dataset['train'].map(preprocess_data, batched=True)
valid_data = dataset['test'].map(preprocess_data, batched=True)

# 建立 DatasetDict
dataset_dict = DatasetDict({
    'train': train_data,
    'test': valid_data
})

# 設定模型評估指標

In [ ]:
# 計算模型評估指標
def compute_metrics(predicted_results):
    # 取得真實標籤
    labels = predicted_results.label_ids

    # 取得預測結果
    preds = predicted_results.predictions.argmax(-1)

    # 計算 F1 score (多元分類使用 weighted)
    # 可參考: https://blog.csdn.net/qq_40671063/article/details/130447922
    f1 = f1_score(labels, preds, average='weighted')
    return {
        'f1': f1,
    }

# 微調模型

In [ ]:
# 讀取模型 (設定 id2label 和 label2id，讓模型知道每個 id 對應的標籤名稱)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
)

# 設定訓練參數
training_args = TrainingArguments(
    output_dir=output_dir, # 輸出資料夾
    overwrite_output_dir=True,
    num_train_epochs=3, # 訓練回合數
    per_device_train_batch_size=32, # 批次大小
    per_device_eval_batch_size=32, # 批次大小
    gradient_accumulation_steps=2, # 梯度累積步數，主要是為了讓較小的 GPU 也能訓練較大的 batch size
    learning_rate=0.00005 * 2, # 預設會將每個小 batch 的 loss 除以累積步數（loss / 2），讓最終梯度等同於單一大 batch，但這會讓梯度變小（等於將 learning rate 除以 2），所以要手動將 learning rate 乘以累積步數來抵消
    warmup_steps=50, # 預熱步數，主要是讓 learning rate 從 0 緩慢增加到設定的 learning rate，避免一開始學習率過大導致模型不穩定
    weight_decay=0.01, # 權重衰減，主要是為了正則化，避免過擬合，讓 loss 曲線較平滑
    eval_strategy="steps", # epoch, steps, no
    eval_steps=70, # 每隔多少步驟評估一次
    save_strategy="steps", # epoch, steps, no
    save_steps=70, # 每隔多少步驟儲存一次
    save_total_limit=2, # 最多儲存模型數量
    load_best_model_at_end=True, # Trainer 會追蹤 metric_for_best_model（預設 eval_loss，越小越好）來判斷最佳模型，訓練結束時自動將該最佳檢查點載入記憶體覆蓋最終模型
    seed=seed, # 隨機種子，主要是為了確保實驗可重現
    lr_scheduler_type="linear", # https://blog.csdn.net/muyao987/article/details/139319466
    report_to="none", # 關閉內建的實驗追蹤功能 (如 wandb, tensorboard)
)

# 設定 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['test'],
    data_collator=None, # DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

# 開始訓練
trainer.train()

# 儲存模型
trainer.save_model(output_dir)

# 儲存 tokenizer
tokenizer.save_pretrained(output_dir)

# 儲存 label 對應表 (方便預測時使用)
import json
with open(f'{output_dir}/label_mapping.json', 'w', encoding='utf-8') as f:
    json.dump({'label2id': label2id, 'id2label': {str(k): v for k, v in id2label.items()}}, f, ensure_ascii=False, indent=2)

print(f"模型已儲存至 {output_dir}")
print(f"Label 對應表已儲存至 {output_dir}/label_mapping.json")